<a href="https://colab.research.google.com/github/VegaSera/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/Wesley_Mountford_LS_DS12_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


# Data Cleaning

In [1]:
import pandas as pd
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [0]:
def yn_to_binary(data):
  if str(data).lower() == 'yes' or str(data).lower() == "male":
    return 1
  elif str(data).lower() == 'no' or str(data).lower() == "female":
    return 0

In [3]:
yes_no_columns = ["Partner", "Dependents", "PhoneService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "PaperlessBilling", "Churn", "gender"]
for col in yes_no_columns:
  df[col] = df[col].apply(yn_to_binary)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0.0,1.0,0.0,0.0,0.0,0.0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1.0,0.0,1.0,0.0,0.0,0.0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1.0,1.0,0.0,0.0,0.0,0.0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1.0,0.0,1.0,1.0,0.0,0.0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0.0,0.0,0.0,0.0,0.0,0.0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,Yes,DSL,1.0,0.0,1.0,1.0,1.0,1.0,One year,1,Mailed check,84.80,1990.5,0
7039,2234-XADUH,0,0,1,1,72,1,Yes,Fiber optic,0.0,1.0,1.0,0.0,1.0,1.0,One year,1,Credit card (automatic),103.20,7362.9,0
7040,4801-JZAZL,0,0,1,1,11,0,No phone service,DSL,1.0,0.0,0.0,0.0,0.0,0.0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,8361-LTMKD,1,1,1,0,4,1,Yes,Fiber optic,0.0,0.0,0.0,0.0,0.0,0.0,Month-to-month,1,Mailed check,74.40,306.6,1


In [4]:
df['PaymentMethod'].value_counts(dropna=False)

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64

In [5]:
!pip install category_encoders
from category_encoders import OneHotEncoder
import numpy as np

encoder = OneHotEncoder(cols=['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod'], handle_unknown='ignore', use_cat_names=True)
df0 = encoder.fit_transform(df)
df0 = df0.drop('customerID', axis=1)

#Nearly empty strings in TotalCharges. Replacing with NaNs
for ind, val in enumerate(df0['TotalCharges']):
  if len(str(val)) <= 1:
    df0['TotalCharges'][ind] = np.NaN
df0['TotalCharges'].astype(float)
df0

     |████████████████████████████████| 102kB 6.9MB/s 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,29.85,29.85,0
1,1,0,0,0,34,1,0,1,0,1,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,0,1,0,0,56.95,1889.5,0
2,1,0,0,0,2,1,0,1,0,1,0,0,1.0,1.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,53.85,108.15,1
3,1,0,0,0,45,0,1,0,0,1,0,0,1.0,0.0,1.0,1.0,0.0,0.0,0,1,0,0,0,0,1,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,0,0,1,1,0,0,1.0,0.0,1.0,1.0,1.0,1.0,0,1,0,1,0,1,0,0,84.80,1990.5,0
7039,0,0,1,1,72,1,0,0,1,0,1,0,0.0,1.0,1.0,0.0,1.0,1.0,0,1,0,1,0,0,0,1,103.20,7362.9,0
7040,0,0,1,1,11,0,1,0,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,29.60,346.45,0
7041,1,1,1,0,4,1,0,0,1,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,74.40,306.6,1


In [6]:
from sklearn.impute import SimpleImputer
nanstuff = ['OnlineSecurity', "OnlineBackup", 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

imr = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
imr = imr.fit(df0[nanstuff])
df0[nanstuff] = imr.transform(df0[nanstuff])
df0.TotalCharges = df0.TotalCharges.fillna(0.0)
#df0 = df0.drop(['MultipleLines_No phone service'], axis=1) # Redundant column with PhoneService
df0
#Data has been cleaned

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,29.85,29.85,0
1,1,0,0,0,34,1,0,1,0,1,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,0,1,0,0,56.95,1889.5,0
2,1,0,0,0,2,1,0,1,0,1,0,0,1.0,1.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,53.85,108.15,1
3,1,0,0,0,45,0,1,0,0,1,0,0,1.0,0.0,1.0,1.0,0.0,0.0,0,1,0,0,0,0,1,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,0,0,1,1,0,0,1.0,0.0,1.0,1.0,1.0,1.0,0,1,0,1,0,1,0,0,84.80,1990.5,0
7039,0,0,1,1,72,1,0,0,1,0,1,0,0.0,1.0,1.0,0.0,1.0,1.0,0,1,0,1,0,0,0,1,103.20,7362.9,0
7040,0,0,1,1,11,0,1,0,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,29.60,346.45,0
7041,1,1,1,0,4,1,0,0,1,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,74.40,306.6,1


In [7]:
for col in df0.columns:
  df0[col] = df0[col].astype(float)
df0.dtypes

gender                                     float64
SeniorCitizen                              float64
Partner                                    float64
Dependents                                 float64
tenure                                     float64
PhoneService                               float64
MultipleLines_No phone service             float64
MultipleLines_No                           float64
MultipleLines_Yes                          float64
InternetService_DSL                        float64
InternetService_Fiber optic                float64
InternetService_No                         float64
OnlineSecurity                             float64
OnlineBackup                               float64
DeviceProtection                           float64
TechSupport                                float64
StreamingTV                                float64
StreamingMovies                            float64
Contract_Month-to-month                    float64
Contract_One year              

In [8]:
df0.isnull().any()

gender                                     False
SeniorCitizen                              False
Partner                                    False
Dependents                                 False
tenure                                     False
PhoneService                               False
MultipleLines_No phone service             False
MultipleLines_No                           False
MultipleLines_Yes                          False
InternetService_DSL                        False
InternetService_Fiber optic                False
InternetService_No                         False
OnlineSecurity                             False
OnlineBackup                               False
DeviceProtection                           False
TechSupport                                False
StreamingTV                                False
StreamingMovies                            False
Contract_Month-to-month                    False
Contract_One year                          False
Contract_Two year   

# Fitting Neural Network

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df0)

In [0]:
target = 'Churn'
x_train = train.drop(target, axis=1)
y_train = train[target]
x_test = test.drop(target, axis=1)
y_test = test[target]

In [11]:
!pip install keras --upgrade

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [21]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

inputs = x_train.shape[1]
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0)
adam = keras.optimizers.Adam(learning_rate=0.01)

def create_model(hidden_activation='relu', optimizer=adam, learning_rate=0.01, num_layers=3, neurons_per_layer=64):
  model = Sequential()
  for layer in range(num_layers):
    if layer == 0:
      model.add(Dense(neurons_per_layer, activation=hidden_activation, input_dim=inputs))
    else:
      model.add(Dense(neurons_per_layer, activation=hidden_activation))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size':[16],
              'epochs':[60],
              'hidden_activation':['selu'],
              'learning_rate':[0.01],
              'num_layers':[1],
              'neurons_per_layer':[60]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7949630498886109 using {'batch_size': 16, 'epochs': 60, 'hidden_activation': 'selu', 'learning_rate': 0.01, 'neurons_per_layer': 60, 'num_layers': 1}
Means: 0.7911767840385437, Stdev: 0.008413807024513385 with: {'batch_size': 16, 'epochs': 60, 'hidden_activation': 'selu', 'learning_rate': 0.01, 'neurons_per_layer': 10, 'num_layers': 1}
Means: 0.7902325034141541, Stdev: 0.01800416275160989 with: {'batch_size': 16, 'epochs': 60, 'hidden_activation': 'selu', 'learning_rate': 0.01, 'neurons_per_layer': 20, 'num_layers': 1}
Means: 0.7881486177444458, Stdev: 0.011409642992326632 with: {'batch_size': 16, 'epochs': 60, 'hidden_activation': 'selu', 'learning_rate': 0.01, 'neurons_per_layer': 30, 'num_layers': 1}
Means: 0.7713113903999329, Stdev: 0.02851656326830618 with: {'batch_size': 16, 'epochs': 60, 'hidden_activation': 'selu', 'learning_rate': 0.01, 'neurons_per_layer': 40, 'num_layers': 1}
Means: 0.7805762767791748, Stdev: 0.0059426741652123275 with: {'batch_size': 16, 'epochs': 6

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?